In [2]:
import csv
import numpy as np
import pandas as pd


1. Выводим prolongations.csv с фильтром по году (нужен 2023) + группируя по id

In [3]:
month_order = {
    'январь': 1, 'февраль': 2, 'март': 3, 'апрель': 4, 'май': 5, 'июнь': 6,
    'июль': 7, 'август': 8, 'сентябрь': 9, 'октябрь': 10, 'ноябрь': 11, 'декабрь': 12
}

df = pd.read_csv('prolongations.csv', encoding='utf-8')

df['year'] = df['month'].str.extract(r'(\d{4})').astype(int)
df['month_name'] = df['month'].str.extract(r'([^\s]+)', expand=False).map(month_order)

temp_df = df[['year', 'month_name']].copy()
temp_df['month'] = temp_df['month_name']
temp_df['day'] = 1
df['date_sort'] = pd.to_datetime(temp_df[['year', 'month', 'day']])

df_2023 = df[df['year'] == 2023].copy()

uniqe_projects = df_2023.groupby('id').agg({
    'month': 'first',
    'AM': 'first',
    'date_sort': 'first'
}).reset_index().sort_values(['date_sort', 'id'], ascending=[True, False])

print(uniqe_projects.shape)
print(uniqe_projects.to_string(index=False))

(259, 4)
  id         month                            AM  date_sort
 732   январь 2023      Кузнецов Михаил Иванович 2023-01-01
 728   январь 2023   Попова Екатерина Николаевна 2023-01-01
 721   январь 2023   Попова Екатерина Николаевна 2023-01-01
 708   январь 2023  Васильев Артем Александрович 2023-01-01
 703   январь 2023  Васильев Артем Александрович 2023-01-01
 697   январь 2023  Васильев Артем Александрович 2023-01-01
 687   январь 2023 Соколова Анастасия Викторовна 2023-01-01
 684   январь 2023  Васильев Артем Александрович 2023-01-01
 674   январь 2023   Попова Екатерина Николаевна 2023-01-01
 671   январь 2023 Соколова Анастасия Викторовна 2023-01-01
 649   январь 2023 Соколова Анастасия Викторовна 2023-01-01
 629   январь 2023      Кузнецов Михаил Иванович 2023-01-01
 624   январь 2023   Попова Екатерина Николаевна 2023-01-01
 548   январь 2023     Михайлов Андрей Сергеевич 2023-01-01
 547   январь 2023     Михайлов Андрей Сергеевич 2023-01-01
 519   январь 2023 Соколова Ана

2. Выводим merged financial.csv с prolongations.csv без дублей + группировка по id

In [4]:
df_fin = pd.read_csv('financial_data.csv', encoding='utf-8')

month_cols = [col for col in df_fin.columns if '2023' in col]
df_fin_2023  = df_fin[['id'] + month_cols + ['Account']].copy()

def parse_amount(val):
    if pd.isna(val):
        return np.nan
    str_val = str(val).strip()
    if str_val in ['стоп', 'end']:
        return np.nan
    if str_val == 'в ноль':
        return 0.0
    
    clean = str_val.replace('\xa0', '').replace(' ', '').replace(',', '.')
    try:
        return float(clean)
    except ValueError:
        return np.nan

for col in month_cols:
    df_fin_2023[col] = df_fin_2023[col].apply(parse_amount)

stop_ids = set(df_fin[df_fin[month_cols].isin(['стоп', 'end']).any(axis=1)]['id'])
df_fin_2023 = df_fin_2023[~df_fin_2023['id'].isin(stop_ids)]

df_fin_grouped = df_fin_2023.groupby('id')[month_cols].sum().reset_index()
df_fin_grouped = df_fin_grouped.dropna(axis=0, how='all')

merged_df = uniqe_projects[uniqe_projects['AM'] != 'без А/М'].merge(df_fin_grouped, on='id', how='left')
merged_df[month_cols] = merged_df[month_cols].fillna(0)

print(merged_df.shape)
print(merged_df.to_string(index=False))

(257, 16)
  id         month                            AM  date_sort  Январь 2023  Февраль 2023  Март 2023  Апрель 2023  Май 2023  Июнь 2023  Июль 2023  Август 2023  Сентябрь 2023  Октябрь 2023  Ноябрь 2023  Декабрь 2023
 732   январь 2023      Кузнецов Михаил Иванович 2023-01-01         0.00          0.00       0.00         0.00 306425.00       0.00       0.00         0.00           0.00          0.00         0.00          0.00
 728   январь 2023   Попова Екатерина Николаевна 2023-01-01         0.00          0.00       0.00         0.00      0.00       0.00       0.00         0.00           0.00          0.00         0.00          0.00
 721   январь 2023   Попова Екатерина Николаевна 2023-01-01     35700.00          0.00       0.00         0.00      0.00       0.00       0.00         0.00           0.00          0.00         0.00          0.00
 708   январь 2023  Васильев Артем Александрович 2023-01-01    660475.00     752515.00  795090.00    683985.00 145465.00  163900.00  144725.00

3. Определяем завершившиеся проекты для расчета коэффициента 

In [ ]:
month_map = {
    'январь 2023': 1, 'февраль 2023': 2, 'март 2023': 3, 'апрель 2023': 4, 'май 2023': 5,
    'июнь 2023': 6, 'июль 2023': 7, 'август 2023': 8, 'сентябрь 2023': 9, 'октябрь 2023': 10,
    'ноябрь 2023': 11, 'декабрь 2023': 12
}

target_months_cap = [
    'Январь 2023', 'Февраль 2023', 'Март 2023', 'Апрель 2023', 'Май 2023',
    'Июнь 2023', 'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023',
    'Ноябрь 2023', 'Декабрь 2023'
]

def get_prev_month(target, steps=1):
    num = month_map.get(target)
    if num is None:
        return None
    prev_num = num - steps
    if prev_num < 1:
        return None
    for m, n in month_map.items():
        if n == prev_num:
            return m
    return None

bases = []

for target in month_map.keys():
    prev1 = get_prev_month(target)
    prev2 = get_prev_month(target, 2)
    if not prev1:
        continue

    prev1_projects = merged_df[merged_df['month'] == prev1]

    idx_prev1 = month_map[prev1] - 1
    col_prev1 = target_months_cap[idx_prev1]
    base1_total = prev1_projects[col_prev1].sum()
    base1_by_am = prev1_projects.groupby('AM')[col_prev1].sum()

    if prev2:
        prev2_projects = merged_df[merged_df['month'] == prev2]
        idx_prev1_for_prev2 = month_map[prev1] - 1
        col_prev1_for_prev2 = target_months_cap[idx_prev1_for_prev2]
        no_prolong1 = prev2_projects[prev2_projects[col_prev1_for_prev2] == 0]
        idx_prev2 = month_map[prev2] - 1
        col_prev2 = target_months_cap[idx_prev2]
        base2_total = no_prolong1[col_prev2].sum()
        base2_by_am = no_prolong1.groupby('AM')[col_prev2].sum()
    else:
        base2_total = 0
        base2_by_am = pd.Series()

    row = {'target_month': target, 'prev1_month': prev1, 'prev2_month': prev2,
           'base1_total': base1_total, 'base2_total': base2_total}
    row.update({'base1_' + str(am): val for am, val in base1_by_am.items()})
    row.update({'base2_' + str(am): val for am, val in base2_by_am.items()})
    bases.append(row)

bases_df = pd.DataFrame(bases)
print(bases_df.to_string(index=False))
bases_df.to_csv('bases_2023.csv', index=False, encoding='utf-8') # csv для отчета

 target_month   prev1_month   prev2_month  base1_total  base2_total  base1_Васильев Артем Александрович  base1_Кузнецов Михаил Иванович  base1_Михайлов Андрей Сергеевич  base1_Попова Екатерина Николаевна  base1_Смирнова Ольга Владимировна  base1_Соколова Анастасия Викторовна  base1_Иванова Мария Сергеевна  base2_Васильев Артем Александрович  base2_Кузнецов Михаил Иванович  base2_Попова Екатерина Николаевна  base2_Смирнова Ольга Владимировна  base2_Соколова Анастасия Викторовна  base2_Иванова Мария Сергеевна  base2_Михайлов Андрей Сергеевич  base1_Петрова Анна Дмитриевна
 февраль 2023   январь 2023          None   2460100.50         0.00                           756225.00                        66255.00                        900420.00                          193877.00                           48000.00                            495323.50                            NaN                                 NaN                             NaN                                NaN              

In [12]:
bases_df = pd.read_csv('bases_2023.csv', encoding='utf-8')

month_map = {
    'январь 2023': 1, 'февраль 2023': 2, 'март 2023': 3, 'апрель 2023': 4, 'май 2023': 5,
    'июнь 2023': 6, 'июль 2023': 7, 'август 2023': 8, 'сентябрь 2023': 9, 'октябрь 2023': 10,
    'ноябрь 2023': 11, 'декабрь 2023': 12
}

target_months_cap = [
    'Январь 2023', 'Февраль 2023', 'Март 2023', 'Апрель 2023', 'Май 2023',
    'Июнь 2023', 'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023',
    'Ноябрь 2023', 'Декабрь 2023'
]

def get_prev_month(target, steps=1):
    num = month_map.get(target)
    if num is None:
        return None
    prev_num = num - steps
    if prev_num < 1:
        return None
    for m, n in month_map.items():
        if n == prev_num:
            return m
    return None

coeffs = []

for target in month_map.keys():
    prev1 = get_prev_month(target)
    prev2 = get_prev_month(target, 2)
    if not prev1:
        continue

    prev1_projects = merged_df[merged_df['month'] == prev1]
    idx_target = month_map[target] - 1
    col_target = target_months_cap[idx_target]
    col_prev1 = target_months_cap[month_map[prev1] - 1]

    prolong1_projects = prev1_projects[prev1_projects[col_target] > 0]
    prolong1_total = prolong1_projects[col_target].sum()
    base1_total = prev1_projects[col_prev1].sum()  # Из bases_df, но здесь пересчитано для консистенции
    коэф1_total = prolong1_total / base1_total if base1_total > 0 else np.nan
    коэф1_by_am = (prolong1_projects.groupby('AM')[col_target].sum() / prev1_projects.groupby('AM')[col_prev1].sum()).fillna(0)
    
    if prev2:
        prev2_projects = merged_df[merged_df['month'] == prev2]
        col_prev1_for_prev2 = target_months_cap[month_map[prev1] - 1]
        no_prolong1 = prev2_projects[prev2_projects[col_prev1_for_prev2] == 0]
        col_prev2 = target_months_cap[month_map[prev2] - 1]
        prolong2_projects = no_prolong1[no_prolong1[col_target] > 0]
        prolong2_total = prolong2_projects[col_target].sum()
        base2_total = no_prolong1[col_prev2].sum()
        коэф2_total = prolong2_total / base2_total if base2_total > 0 else np.nan
        коэф2_by_am = (prolong2_projects.groupby('AM')[col_target].sum() / no_prolong1.groupby('AM')[col_prev2].sum()).fillna(0)
    else:
        коэф2_total = np.nan
        коэф2_by_am = pd.Series()
        base2_total = 0

    row = {'target_month': target, 'prev1_month': prev1, 'prev2_month': prev2,
           'коэф1_total': коэф1_total, 'коэф2_total': коэф2_total,
           'base1_total': base1_total, 'base2_total': base2_total}
    row.update({'коэф1_' + str(am): val for am, val in коэф1_by_am.items()})
    row.update({'коэф2_' + str(am): val for am, val in коэф2_by_am.items()})
    coeffs.append(row)

coeffs_df = pd.DataFrame(coeffs)

#За год: взвешенный средний
if not coeffs_df.empty:
    total_base1 = coeffs_df['base1_total'].sum()
    coeffs_df['коэф1_year'] = (coeffs_df['коэф1_total'] * coeffs_df['base1_total']).sum() / total_base1 if total_base1 > 0 else np.nan
    
    total_base2 = coeffs_df['base2_total'].sum()
    coeffs_df['коэф2_year'] = (coeffs_df['коэф2_total'] * coeffs_df['base2_total']).sum() / total_base2 if total_base2 > 0 else np.nan

    am_list = ['Васильев Артем Александрович', 'Кузнецов Михаил Иванович', 'Михайлов Андрей Сергеевич', 'Попова Екатерина Николаевна', 'Смирнова Ольга Владимировна', 'Соколова Анастасия Викторовна', 'Иванова Мария Сергеевна', 'Петрова Анна Дмитриевна']
    for am in am_list:
        base1_col = 'base1_' + am
        коэф1_col = 'коэф1_' + am
        if base1_col in coeffs_df.columns and коэф1_col in coeffs_df.columns:
            total_am_base1 = coeffs_df[base1_col].sum()
            coeffs_df[f'{коэф1_col}_year'] = (coeffs_df[коэф1_col] * coeffs_df[base1_col]).sum() / total_am_base1 if total_am_base1 > 0 else np.nan

print(coeffs_df.to_string(index=False))
coeffs_df.to_csv('coeffs_2023.csv', index=False, encoding='utf-8')

 target_month   prev1_month   prev2_month  коэф1_total  коэф2_total  base1_total  base2_total  коэф1_Васильев Артем Александрович  коэф1_Кузнецов Михаил Иванович  коэф1_Михайлов Андрей Сергеевич  коэф1_Попова Екатерина Николаевна  коэф1_Смирнова Ольга Владимировна  коэф1_Соколова Анастасия Викторовна  коэф1_Иванова Мария Сергеевна  коэф2_Васильев Артем Александрович  коэф2_Кузнецов Михаил Иванович  коэф2_Попова Екатерина Николаевна  коэф2_Смирнова Ольга Владимировна  коэф2_Соколова Анастасия Викторовна  коэф2_Иванова Мария Сергеевна  коэф2_Михайлов Андрей Сергеевич  коэф1_Петрова Анна Дмитриевна  коэф1_year  коэф2_year
 февраль 2023   январь 2023          None     0.764581          NaN   2460100.50         0.00                            1.155761                        0.846955                         0.903645                           0.000000                           0.000000                             0.276900                            NaN                                 NaN     